<h1>SD-WAN Basic Automation Bootcamp</h1>
<p>
The goal of this notebook is to get familar with working with the sd-wan apis
</p>
<ol>
    <li>Authenticate</li>
    <li>Get all devices in overlay</li>
    <li>Get all device templates</li>
    <li>Get Real Time BFD Sessions for a specic device</li>
</ol>
<h3>Import Libraries</h3>

In [ ]:
"""
copyright Cisco
created by bshokrol
04/24/2021

"""

#pip install requests
import requests
#ignore insecure requests warning
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


import getpass
import sys
import json
#pip install pandas
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 2000)

<h3>Create class</h3>

In [ ]:
class SdwanBootcamp:
    def __init__(self, vmanage_address: str, vmanage_username: str, vmanage_password: str, verify: bool = False):
        self.session = self.authenticate(vmanage_address, vmanage_username, vmanage_password)
        self.vmanage_address = vmanage_address
        self.verify = verify

    @staticmethod
    def authenticate(vmanage_address: str, vmanage_username: str, vmanage_password: str ,verify: bool = False) -> requests.session:
        session = requests.session()
        payload = {
            "j_username": vmanage_username,
            "j_password": vmanage_password
        }
        url = f"{vmanage_address}j_security_check"
        response = session.post(url=url, data=payload, verify=verify)
        if "<html>" in str(response.content):
            print("Authentication failed")
            sys.exit(0)
        print("Authenticated")
        session.headers["X-XSRF-TOKEN"] = session.get(f"{vmanage_address}dataservice/client/token").content
        session.headers["Content-Type"] = "application/json"
        return session

    def post_request(self, endpoint: str, payload: dict) -> requests.Response:
        data = json.dumps(payload)
        url = f"{self.vmanage_address}{endpoint}"
        return self.session.post(url=url, data=data, verify=self.verify)

    def get_request(self, endpoint: str) -> requests.Response:
        url = f"{self.vmanage_address}{endpoint}"
        return self.session.get(url=url, verify=self.verify)


<h3>1. Initialize class with credentials and Authenticate</h3>

In [ ]:
vmanage_ip = "128.107.222.2"
vmanage_port = "8443"
vmanage_username = "admin"
vmanage_password = getpass.getpass("vmanage password: ")
vmanage_address = f"https://{vmanage_ip}:{vmanage_port}/"
sdwan_bootcamp = SdwanBootcamp(vmanage_address=vmanage_address, vmanage_username=vmanage_username, vmanage_password=vmanage_password)

<h3>2. Get all devices in overlay using get_request</h3>
<a href="https://developer.cisco.com/docs/sdwan/#!device-inventory">Get Connected Devices</a>

In [ ]:
get_devices = "dataservice/device"
response = sdwan_bootcamp.get_request(endpoint=get_devices)
print(response.status_code)
devices_df = pd.DataFrame(response.json().get("data"))

In [ ]:
devices_df

<h3>3. Get all device templates in vmanage using get_request</h3>
<a href="https://developer.cisco.com/docs/sdwan/#!device-configuration">Get Device Templates</a>


In [ ]:
get_all_device_templates = "dataservice/template/device"
response = sdwan_bootcamp.get_request(get_all_device_templates)
print(response.status_code)
all_device_templates_df = pd.DataFrame(response.json().get("data"))

In [ ]:
all_device_templates_df

<h3>4. Get current bfd session for a specific device using get_request</h3>
<a href="https://developer.cisco.com/docs/sdwan/#!device-realtime-monitoring/bfd">Real time monitoring BFD</a>


In [ ]:
get_bfd_sess = "dataservice/device/bfd/sessions?deviceId={0}"
response = sdwan_bootcamp.get_request(get_bfd_sess.format("1.1.1.15"))
print(response.status_code)
bfd_sess_df = pd.DataFrame(response.json().get("data"))

In [ ]:
bfd_sess_df